# OpenAI Models on Amazon Bedrock

This notebook demonstrates the basic functionality of using OpenAI models on Amazon Bedrock through different authentication methods and API patterns.

## What You'll Learn
1. **Generate short-term Bedrock API keys** for OpenAI compatibility
2. **Use OpenAI SDK** with Bedrock endpoints
3. **Direct Bedrock InvokeModel** API calls
4. **Bedrock Converse API** for multi-turn conversations

## Prerequisites
- AWS credentials configured
- Access to Amazon Bedrock in us-west-2
- OpenAI model (gpt-oss-20b) access enabled

## Setup and Constants

First, let's import the required libraries and set up our constants.

In [2]:
# Import required libraries
import boto3
import json
import base64
import openai
from datetime import timedelta
from aws_bedrock_token_generator import provide_token

# Constants
REGION = "us-west-2"  # AWS region for Bedrock
MODEL_ID = "openai.gpt-oss-20b-1:0"  # OpenAI model on Bedrock

# Sample automotive diagnostic scenario
VEHICLE_DATA = {
    "make": "Toyota",
    "model": "Camry", 
    "year": 2020,
    "mileage": 45000,
    "symptoms": ["engine noise during acceleration", "reduced fuel efficiency"],
    "error_codes": ["P0171", "P0174"]
}

print(f"Region: {REGION}")
print(f"Model: {MODEL_ID}")
print(f"Sample Vehicle: {VEHICLE_DATA['year']} {VEHICLE_DATA['make']} {VEHICLE_DATA['model']}")

Region: us-west-2
Model: openai.gpt-oss-20b-1:0
Sample Vehicle: 2020 Toyota Camry


## 1. Generate Short-Term Bedrock API Key

The first step is to generate a temporary API key that allows OpenAI SDK compatibility with Bedrock. This key:
- Expires in 1 hour for security
- Is based on your current AWS role/credentials
- Contains a base64-encoded presigned URL for authentication

In [ ]:
# Get current AWS identity
sts_client = boto3.client('sts', region_name=REGION)
identity = sts_client.get_caller_identity()
role_arn = identity['Arn']



# Generate short-term API key with explicit 1-hour expiry
api_key = provide_token(
    region=REGION,
    expiry=timedelta(hours=1)
)

# Mask API key for security - only show last 8 characters
masked_key = "XXX..." + api_key[-8:] if len(api_key) > 8 else "XXX..."


# Decode and analyze the token structure for educational purposes
prefix = "bedrock-api-key-"
if api_key.startswith(prefix):
    encoded_part = api_key[len(prefix):]
    
    
# Decode the base64 to get the presigned URL
decoded_url = base64.b64decode(encoded_part).decode("utf-8")

In [ ]:
#print(decoded_url)

## 2. OpenAI SDK with Bedrock

Now let's use the generated API key with the OpenAI SDK, configured to point to Bedrock endpoints. This allows you to use familiar OpenAI patterns while leveraging Bedrock's infrastructure.

In [5]:
# Configure OpenAI client for Bedrock with correct endpoint
client = openai.OpenAI(
    base_url=f"https://bedrock-runtime.{REGION}.amazonaws.com/openai/v1",
    api_key=api_key
)

# Create a diagnostic prompt for our automotive scenario
diagnostic_prompt = f"""You are a senior automotive diagnostic technician. Analyze this vehicle issue:

Vehicle: {VEHICLE_DATA['year']} {VEHICLE_DATA['make']} {VEHICLE_DATA['model']}
Mileage: {VEHICLE_DATA['mileage']} miles
Symptoms: {', '.join(VEHICLE_DATA['symptoms'])}
Error Codes: {', '.join(VEHICLE_DATA['error_codes'])}

Please provide:
1. Analysis of the problem
2. Possible causes
3. Recommended actions
4. Urgency level (low/medium/high)"""

print("OpenAI Chat Completion with Bedrock")
print("Sending diagnostic request...")

# Send chat completion request
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {"role": "user", "content": diagnostic_prompt}
    ],
    max_completion_tokens=500,
    temperature=0.7
)

# Display response
print("\nDiagnostic Analysis:")
print("=" * 50)
if response.choices and response.choices[0].message:
    print(response.choices[0].message.content)
    
    if response.usage:
        print(f"\n Tokens used: {response.usage.total_tokens}")
else:
    print(f"Full response: {response}")

OpenAI Chat Completion with Bedrock
Sending diagnostic request...

Diagnostic Analysis:
<reasoning>The user wants a diagnostic analysis: 2020 Toyota Camry, 45,000 miles, engine noise during acceleration, reduced fuel efficiency. Error codes P0171 (System too lean: bank 1) and P0174 (System too lean: bank 2). Provide analysis, possible causes, recommended actions, urgency level. Must consider typical 2020 Camry. Could be 2.5L 4-cylinder or 3.5L V6. Likely 2.5L. Engine noise during acceleration could be misfire, or low timing, or EGR, or wrong fuel pressure. Reduced fuel efficiency plus lean codes indicates low fuel or high air. Could be vacuum leak, clogged MAP, dirty idle air control, throttle body, or mass airflow sensor reading incorrectly. Also could be fuel pump or fuel filter.

We need to cover the "engine noise" - misfire, knocking, rough idle. Could be due to lean mixture causing misfires. The codes indicate lean mixture on both banks. Could be due to vacuum leaks, intake manifo

## 3. Direct Bedrock InvokeModel API

This approach uses the native AWS Bedrock API directly with boto3, bypassing the OpenAI SDK. It uses your AWS credentials directly and demonstrates the raw Bedrock API.

In [6]:
# Initialize Bedrock runtime client
bedrock_client = boto3.client('bedrock-runtime', region_name=REGION)

print("AWS Bedrock InvokeModel API")
print("\n Sending diagnostic request...")

# Prepare request body in OpenAI format (Bedrock supports OpenAI format for these models)
request_body = {
    "messages": [
        {"role": "user", "content": diagnostic_prompt}
    ],
    "max_completion_tokens": 500,
    "temperature": 0.7
}

# Invoke model using native Bedrock API
response = bedrock_client.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body),
    contentType='application/json'
)

# Parse response
response_body = json.loads(response['body'].read())

# Display response
print("\n Diagnostic Analysis:")
print("=" * 50)
if 'choices' in response_body and response_body['choices']:
    print(response_body['choices'][0]['message']['content'])
    
    if 'usage' in response_body:
        print(f"\n Tokens used: {response_body['usage']['total_tokens']}")
else:
    print(json.dumps(response_body, indent=2))

AWS Bedrock InvokeModel API

 Sending diagnostic request...

 Diagnostic Analysis:
<reasoning>We need to respond as a senior automotive diagnostic technician. The vehicle is a 2020 Toyota Camry, 45k miles. Symptoms: engine noise during acceleration, reduced fuel efficiency. Error codes: P0171 (System too lean, Bank 1) and P0174 (System too lean, Bank 2). Provide analysis, possible causes, recommended actions, urgency level.

We should mention that P0171/4 indicate lean condition. Engine noise during acceleration could be due to misfire or rough idle, but lean condition can cause increased exhaust temperatures, misfire, and noise. Reduced fuel efficiency matches lean mixture.

Possible causes: vacuum leaks, faulty MAF or MAP sensor, throttle body, idle air control valve, fuel injector problem, fuel pressure regulator, faulty fuel pump, bad O2 sensor, poor fuel quality, evaporative emissions system leak, throttle position sensor, air intake system. Also could be due to incorrect fuel tri

## 4. Bedrock Converse API

The Converse API is Bedrock's native conversation interface that supports multi-turn dialogues. It's designed for building conversational applications and maintains context across turns.

In [7]:
print("AWS Bedrock Converse API")
print("Sending diagnostic request...")

# First conversation turn - diagnostic analysis
response = bedrock_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": diagnostic_prompt}]
        }
    ],
    inferenceConfig={
        "maxTokens": 500,
        "temperature": 0.7
    }
)

# Display first response
print("\nInitial Diagnostic Analysis:")
print("=" * 50)

if 'output' in response and 'message' in response['output']:
    content = response['output']['message']['content']
    # Find the text content (skip reasoning content)
    for item in content:
        if 'text' in item and item['text'].strip():
            first_response = item['text']
            print(first_response)
            break
    else:
        # If no text content, try reasoning content
        for item in content:
            if 'reasoningContent' in item and 'reasoningText' in item['reasoningContent']:
                reasoning_text = item['reasoningContent']['reasoningText']['text']
                print(f"[Model reasoning]: {reasoning_text[:300]}...")
                first_response = "I analyzed your vehicle's diagnostic codes and symptoms."
                break
        else:
            print("No displayable content found")
else:
    print("Unexpected response format")

# Second conversation turn - follow-up question
follow_up_question = "What would be the estimated cost to fix these issues?"

print(f"\n Follow-up Question: {follow_up_question}")

response = bedrock_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user", 
            "content": [{"text": diagnostic_prompt}]
        },
        {
            "role": "assistant",
            "content": [{"text": first_response}]
        },
        {
            "role": "user",
            "content": [{"text": follow_up_question}]
        }
    ],
    inferenceConfig={
        "maxTokens": 300,
        "temperature": 0.7
    }
)

# Display follow-up response
print("\nCost Estimate:")
print("=" * 50)
content = response['output']['message']['content']
# Find the text content (skip reasoning content)
for item in content:
    if 'text' in item:
        print(item['text'])
        break

print(f"\nTokens used: {response['usage']['totalTokens']}")

AWS Bedrock Converse API
Sending diagnostic request...

Initial Diagnostic Analysis:
[Model reasoning]: We need to respond as a senior automotive diagnostic technician, analyze the issue: 2020 Toyota Camry, 45k miles, symptoms: engine noise during acceleration, reduced fuel efficiency, error codes P0171 (System too lean (Bank 1)), P0174 (System too lean (Bank 2)). Provide analysis of problem, possible...

 Follow-up Question: What would be the estimated cost to fix these issues?

Cost Estimate:
Below is a **ball‑park cost guide** for the most common repairs that would address the P0171/P0174 (lean‑bank) codes and the “engine noise during acceleration” symptom.  
All figures are **range estimates** (parts + labor) for a 2020 Toyota Camry (1.5‑L or 2.5‑L 4‑cyl). Exact costs will vary by shop, region, and whether the problem is a single component or a multi‑step fix.

| # | Likely Fix | Typical Parts | Typical Labor | Total Estimate |
|---|------------|---------------|---------------|----

### Produce Structured Output

In [26]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a maintenance assistant. Provide vehicle diagnostic details in the following structured JSON format: "
            '{"vehicle_id": "<string>", "error_code": "<string>", "problem": "<string>", "recommended_action": "<string>"}'
        )
    },
    {
        "role": "user",
        "content": (
            "Vehicle: Honda Accord 2018\nError code: P0401\n"
            "Describe the issue and recommended action in the JSON format above."
        )
    }
]

payload = {
    "model": MODEL_ID,  
    "messages": messages,
    "max_completion_tokens": 512,
    "temperature": 0.2,
}

body_bytes = json.dumps(payload).encode("utf-8")

response = bedrock_client.invoke_model(
    modelId=MODEL_ID,
    body=body_bytes
)

result = response['body'].read().decode()
print(result)

{"id":"chatcmpl-70fa5d36","choices":[{"finish_reason":"stop","index":0,"message":{"content":"<reasoning>We need to respond with JSON: {\"vehicle_id\": \"<string>\", \"error_code\": \"<string>\", \"problem\": \"<string>\", \"recommended_action\": \"<string>\"}. Provide vehicle_id: maybe \"Honda Accord 2018\" or some ID. Use vehicle_id: \"Honda Accord 2018\". error_code: \"P0401\". problem: description of issue. recommended_action: recommended action. Provide in JSON. Ensure valid JSON.</reasoning>{\"vehicle_id\":\"Honda Accord 2018\",\"error_code\":\"P0401\",\"problem\":\"The engine control module (ECM) has detected a malfunction in the Exhaust Gas Recirculation (EGR) system, indicating that the EGR valve is not functioning properly or the EGR control circuit is not operating within the expected parameters.\",\"recommended_action\":\"Inspect the EGR valve for proper operation, clean or replace it if clogged or stuck. Check the EGR valve solenoid and associated wiring for damage or poor 

In [27]:
reason_token_end = '</reasoning>'
message = json.loads(result)
message = message['choices'][0]['message']['content']
message = message[message.find(reason_token_end) + len(reason_token_end):]
message = json.loads(message)
print(message)

{'vehicle_id': 'Honda Accord 2018', 'error_code': 'P0401', 'problem': 'The engine control module (ECM) has detected a malfunction in the Exhaust Gas Recirculation (EGR) system, indicating that the EGR valve is not functioning properly or the EGR control circuit is not operating within the expected parameters.', 'recommended_action': 'Inspect the EGR valve for proper operation, clean or replace it if clogged or stuck. Check the EGR valve solenoid and associated wiring for damage or poor connections. Verify the EGR temperature sensor and related sensors. After repairs, clear the fault code and perform a drive cycle to ensure the issue is resolved.'}


## Summary

In this notebook, we demonstrated four different ways to use OpenAI models on Amazon Bedrock:

1. **🔑 API Key Generation**: Created temporary credentials for OpenAI SDK compatibility
2. **🔧 OpenAI SDK**: Used familiar OpenAI patterns with Bedrock endpoints
3. **⚡ InvokeModel API**: Direct Bedrock API calls with AWS credentials
4. **💬 Converse API**: Native Bedrock conversation interface with multi-turn support

### Key Takeaways:
- **Flexibility**: Multiple authentication and API patterns available
- **Security**: Short-term tokens and AWS credential-based access
- **Compatibility**: OpenAI SDK works seamlessly with Bedrock
- **Native Features**: Bedrock Converse API provides advanced conversation capabilities

### Next Steps:
- Explore the advanced agent implementations in the next notebook
- Try different OpenAI models available on Bedrock
- Implement your own use cases using these patterns